In [1]:
import pandas as pd
df = pd.read_csv('../../data.csv', parse_dates=['arrival_planned_time', 'departure_planned_time'])
df.head()
#df.dtypes

,station,train_name,final_destination_station,arrival_planned_time,arrival_time_delta_in_min,departure_planned_time,departure_time_delta_in_min,planned_platform,changed_platform,is_canceled,train_type,train_line_ride_id,train_line_station_num
0,Aachen Hbf,RE 18,Aachen Hbf,2024-05-10 00:45:00,6.0,NaT,0.0,1,1,False,RE,-5376151638251295800-2405092354,9
1,Aachen Hbf,RB 33,Aachen Hbf,2024-05-10 00:27:00,0.0,NaT,0.0,2,6,False,RB,3877311608492123870-2405092216,31
2,Aachen Hbf,RE 18,Heerlen,NaT,0.0,2024-05-10 00:17:00,0.0,6,NaN,False,RE,349781417030375472-2405100017,1
3,Aachen Hbf,RE 4,Aachen Hbf,2024-05-10 00:37:00,6.0,NaT,0.0,6,NaN,False,RE,-2351366154733574977-2405092209,24
4,Aachen Hbf,RE 9,Aachen Hbf,2024-05-10 00:41:00,17.0,NaT,0.0,3,9,False,RE,-1701587498056121817-2405092210,20


In [2]:
df_local = df[df["train_type"].isin(["RE", "RB", "S"])]
train_in_dortmund = df_local[df_local["station"] == "Dortmund Hbf"]
train_in_duesseldorf = df_local[df_local["station"] == "Düsseldorf Hbf"]

In [3]:
# Merge/join the two DataFrames on 'train_line_ride_id' to include necessary time columns
merged_df = train_in_duesseldorf.merge(
    train_in_dortmund[['train_line_ride_id', 'train_line_station_num', 'departure_planned_time', 'arrival_planned_time']],
    on='train_line_ride_id',
    suffixes=('_duesseldorf', '_dortmund')
)

In [4]:
# Calculate the planned trip time between Dortmund and Düsseldorf
merged_df['planned_trip_time_in_min'] = (merged_df['arrival_planned_time_duesseldorf'] - merged_df['departure_planned_time_dortmund']).dt.total_seconds() / 60

# only get trains going from Dortmund to Düsseldorf
filtered_df = merged_df[merged_df["planned_trip_time_in_min"] > 0]


In [5]:
#filtered_df = filtered_df[~filtered_df["is_canceled"]]

In [6]:
# Group by train_name and calculate mean, median, and count for each relevant column
trip_stats = filtered_df.groupby('train_name').agg({
    'planned_trip_time_in_min': ['mean'],
    'arrival_time_delta_in_min': ['mean', 'count'],
    'is_canceled': 'mean'  # mean here calculates the percentage of cancellations
}).reset_index()

# flatten columns
trip_stats.columns = ['_'.join(col).strip() if (col[1] and len(col[1]) > 0) else col[0] for col in trip_stats.columns.values]

rename_columns = {
    'planned_trip_time_in_min_mean': 'avg plan trip time [min]',
    'arrival_time_delta_in_min_mean': 'avg delay [min]',
    'arrival_time_delta_in_min_count': 'sample size',
    'is_canceled_mean': 'is canceled [%]'
}
trip_stats = trip_stats.rename(columns=rename_columns)

# filter out trains_names with only a few data points
#trip_stats = trip_stats[trip_stats["sample size"] > 20]

# Convert the cancellation percentage to a proper percentage value
trip_stats['is canceled [%]'] *= 100

# Round columns to integer values
columns_to_round = ['avg plan trip time [min]', 'avg delay [min]', 'is canceled [%]']
trip_stats[columns_to_round] = trip_stats[columns_to_round].round(0).astype(int)

trip_stats["final avg travel time [min]"] = trip_stats["avg plan trip time [min]"] + trip_stats["avg delay [min]"]

# Sort the DataFrame by the total average travel time in ascending order
trip_stats = trip_stats.sort_values(by='final avg travel time [min]', ascending=True)


# Reindex the DataFrame with the new column order
trip_stats = trip_stats[['train_name', 'final avg travel time [min]', 'avg plan trip time [min]', 'avg delay [min]', 'is canceled [%]', 'sample size']]

# View the reordered DataFrame
trip_stats

,train_name,final avg travel time [min],avg plan trip time [min],avg delay [min],is canceled [%],sample size
0,RE 1,61,53,8,3,541
4,RE 6,67,58,9,2,471
1,RE 11,78,63,15,22,45
3,RE 4,81,69,12,5,401
2,RE 3,84,73,11,8,498
5,S 1,98,93,5,7,949


,train_name,final avg travel time [min],avg plan trip time [min],avg delay [min],is canceled [%],sample size
0,RE 1,62,53,9,5,166
4,RE 6,70,58,12,5,145
3,RE 4,80,69,11,6,121
2,RE 3,84,73,11,7,147
5,S 1,102,94,8,19,281
